In [ ]:
# Imports
import os
import sys
import math
import time
import numpy as np
import networkx as nx
import plotly.express as px
import plotly.graph_objects as go

# Ensure notebook is running from Tournesol-Stats dir
_pwd = os.path.realpath('.').split(os.sep)
if 'src' in _pwd:
	while _pwd[-1] != 'src':
		_pwd.pop()
	_pwd.pop() # Go up from src dir to Tournesol-Stats
	os.chdir(os.sep.join(_pwd))
print(os.path.realpath('.'))

# Local project requirements
sys.path.append('src/py')
from scripts.nxlayouts import radialized_layout
from dao.tournesol_api import TournesolCommonAPI, TournesolUserAPI, get, get_individual_score

# Graph

Draw publicaly compared videos (as colored circles) and the comparisons between them.

Positions does not have meaning. Videos move around to try to untangle the graph as much as possible (compared videos nearer, and others further away).

Color Legend:
- Dark Blue: Central videos (with few jumps from comparison to comparison, rapidly access every other videos)
- Blue: Well connected videos
- Green~Yellow: Average
- Orange: Distant videos, more comparisons may be needed
- Red: Most distant videos (Most of the others videos have a high number of jumps from comparison to comparison to get to it)

Note: Distante calculation (& color) take into account private comparisons, even if private videos arn't displayed (a blue or green node may be rendered far away from the center for example if it is compared with 1 comparison to a private video connected to a central video)

In [ ]:
# PARAMETERS
TOURNESOL_API=TournesolUserAPI(TournesolCommonAPI('./data/TournesolAPI_cache'), input('JWT (example: "Bearer xxxxxxxxx")'))
TOURNESOL_API.loadCache()

In [ ]:
# Load dataset
TOURNESOL_API.getMyComparedVideos()
comparisons = TOURNESOL_API.getAllMyComparisons()

private_digraph = nx.DiGraph()
for cdata in comparisons:
	score = [dta['score'] for dta in cdata['criteria_scores'] if dta['criteria'] == 'largely_recommended'][0]
	
	s = abs(score) # Score: from 0 to 10
	w = 1 if s == 0 else 1/(s+1) # Attractive force from 0 to 1: Larger score => lower value

	if score >= 0:
		private_digraph.add_edge(cdata['entity_a'], cdata['entity_b'], score=s, weight=w)
	if score <= 0:
		private_digraph.add_edge(cdata['entity_b'], cdata['entity_a'], score=s, weight=w)

videos = {vid: TOURNESOL_API.getVData(vid, useCache=True, saveCache=False) for vid in private_digraph.nodes}
TOURNESOL_API.saveCache()

for vid in videos:
	private_digraph.nodes[vid]['public'] = get(videos[vid], False, 'individual_rating', 'is_public')

private_undgraph = private_digraph.to_undirected(as_view=True)
print('Videos', len(videos))
print('Comparisons', len(comparisons))
print('Private', private_undgraph)
print('Directed', private_digraph)

MAX_CONNECTED_GRAPH = private_undgraph.subgraph(max(nx.connected_components(private_undgraph), key=len))

## 2D Graph

In [ ]:
# Display graph
def pos_to_graphlocs(graph:nx.Graph, pos:dict[str,list[int]]):
	nodes = {'x': [], 'y': [], 'l': []}
	for node in graph:
		if node in pos:
			x, y = pos[node]
			nodes['l'].append(node)
			nodes['x'].append(x)
			nodes['y'].append(y)

	edges = {'x': [], 'y': []}
	for edge in graph.edges():
		if not (edge[0] in pos and edge[1] in pos): continue
		x0, y0 = pos[edge[0]]
		x1, y1 = pos[edge[1]]
		
		if len(edges['x']) > 2 and edges['x'][-2] == x0 and edges['y'][-2] == y0:
			edges['x'].insert(-1, x1)
			edges['y'].insert(-1, y1)
		elif len(edges['x']) > 2 and edges['x'][-2] == x1 and edges['y'][-2] == y1:
			edges['x'].insert(-1, x0)
			edges['y'].insert(-1, y0)
		else:
			edges['x'].append(x0)
			edges['x'].append(x1)
			edges['y'].append(y0)
			edges['y'].append(y1)
			edges['x'].append(None)
			edges['y'].append(None)

	return {
		'nodes': nodes,
		'edges': edges
	}

def init_comparisons_graph(graph:nx.Graph):
	pos = radialized_layout(MAX_CONNECTED_GRAPH, pos=nx.circular_layout(MAX_CONNECTED_GRAPH))
	loc = pos_to_graphlocs(MAX_CONNECTED_GRAPH, pos)

	scatters = []
	# edges
	scatters.append(go.Scatter(
		x=loc['edges']['x'], y=loc['edges']['y'],
		line=dict(
			width=0.3,
			color='#888',
		),
		hoverinfo='none',
		mode='lines',
		showlegend=False,
	))

	# Nodes text & colors
	cnct:dict[str,float] = dict()
	mx = MAX_CONNECTED_GRAPH.number_of_nodes()
	for n1,tgt in nx.all_pairs_shortest_path_length(MAX_CONNECTED_GRAPH, cutoff=32):
		ttl = mx
		for n2,ln in tgt.items():
			if n2 == n1: continue
			ttl -= 1/ln
		cnct[n1] = mx/(mx - ttl) # Color: Average distance to every other nodes of the graph

	node_colors = []
	node_text = []
	for node in loc['nodes']['l']:
		deg = MAX_CONNECTED_GRAPH.degree[node]
		node_text.append(
			f"{node}<br>"
			+ f"{deg} comparisons<br>"
			+ f"Average distance: {cnct[node]:.1f}"
		)
		if graph.nodes[node]['public']:
			node_colors.append(cnct[node])

	t_min = int(min(node_colors)+1)
	t_max = int(max(node_colors)+1)
	ticks = list(range(t_min, t_max, int((t_max-t_min)/10) or (t_max-t_min)/10))
	ticks[0] = min(node_colors)
	ticks[-1] = max(node_colors)

	private_locs = {'x':[], 'y':[]}
	public_locs = {'x':[], 'y':[]}
	for i,n in enumerate(loc['nodes']['l']):
		collection = public_locs if graph.nodes[n]['public'] else private_locs
		collection['x'].append(loc['nodes']['x'][i])
		collection['y'].append(loc['nodes']['y'][i])

	# Private nodes
	"""scatters.append(go.Scatter(
		x=private_locs['x'], y=private_locs['y'],
		mode='markers',
		hoverinfo='text',
		marker=dict(
			colorscale='Portland',
			reversescale=False,
			size=1.5,
			line=dict(width=0),
			color=node_colors,
		),
		text=node_text,
		showlegend=False,
	))"""

	# Public nodes
	scatters.append(go.Scatter(
		x=public_locs['x'], y=public_locs['y'],
		mode='markers',
		hoverinfo='text',
		marker=dict(
			colorscale='Portland',
			colorbar=dict(
				title='Avg. distance<br>to others',
				tickformat='.1f',
				tickvals=ticks,
				ticks='outside',
				tickmode='array',
			),
			reversescale=False,
			size=3,
			line=dict(width=0),
			color=node_colors,
		),
		text=node_text,
		showlegend=True,
	))

	fig = go.FigureWidget(data=scatters,
		layout=go.Layout(
			plot_bgcolor='white',
			showlegend=False,
			hovermode='closest',
			margin=dict(b=0,l=0,r=0,t=0),
			xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
			yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
			height=720,
			width=1080,
		),
	)

	# Fix aspect ratio
	fig.update_yaxes(scaleanchor="x", scaleratio=1)

	return fig,pos

fig,pos = init_comparisons_graph(private_undgraph)
fig # Display graph (should be the last line of the notebook cell)

In [ ]:
# Untangle graph above (may run multiple time if needed - Takes time)
TIME_TO_RUN = 180 # Seconds, the longer the prettier
STEP_TIME = TIME_TO_RUN/10 # Seconds

def improve_graph_pos(time_to_run:int, pos:dict[str,list[float]], callback=None, target_refresh_interval:int=None):
	start = time.time()

	target_total_it_count=math.ceil(time_to_run/target_refresh_interval) if target_refresh_interval > 0 else 10
	iterations_count=1
	total_iterations=0
	loops_count = 0
	#layout = ForceLayout(MAX_CONNECTED_GRAPH)
	#layout.update_graph(pos, edge_lengths=lambda e: 1)
	timer_a = time.time()
	while timer_a - start < time_to_run:
		loops_count += 1
		# Move nodes towards eachother if connected, move them apart from eachother if not connected
		pos = nx.spring_layout(MAX_CONNECTED_GRAPH, pos=pos, iterations=iterations_count, center=[0,0])
		#layout.iterate2(repulsion_factor=0.01, repulse_upper_bound=3, inertia_factor=.9)
		total_iterations += iterations_count
		timer_b = time.time()
		speed = iterations_count / (timer_b-timer_a)
		expected_remaining_iterations = speed * (time_to_run - timer_b + start)
		if callback:
			#pos = layout.get_pos()
			callback(pos)
		print(f"Iterations: {total_iterations}/{total_iterations + expected_remaining_iterations:.0f} -- Time: {timer_b-start:.1f}/{time_to_run}s -- Speed: {speed:.1f}/s")
		next_iteration_count = int(math.ceil(expected_remaining_iterations / (target_total_it_count - loops_count if loops_count < target_total_it_count else 1)))
		if loops_count > target_total_it_count or next_iteration_count > iterations_count*2 and loops_count > 1:
			# Spring Layout may stop iterating if found an equilibrium. Try to detect this event and stop before max_duration
			break
		# Prepare next iteration
		iterations_count = next_iteration_count
		timer_a = timer_b

	return pos
	#return layout.get_pos()

def onupdate(pos):
	loc = pos_to_graphlocs(private_undgraph, pos)
	with fig.batch_update():
		fig.data[0]['x'] = loc['edges']['x']
		fig.data[0]['y'] = loc['edges']['y']
		
		# Private
		"""
		zipped = zip(loc['nodes']['l'], loc['nodes']['x'], loc['nodes']['y'])
		prv_l,prv_x,prv_y = zip(*filter(lambda z:not private_undgraph.nodes[z[0]]['public'], zipped))
		fig.data[1]['x'] = prv_x
		fig.data[1]['y'] = prv_y
		"""
		# Public
		zipped = zip(loc['nodes']['l'], loc['nodes']['x'], loc['nodes']['y'])
		pub_l,pub_x,pub_y = zip(*filter(lambda z:private_undgraph.nodes[z[0]]['public'], zipped))
		fig.data[1]['x'] = pub_x
		fig.data[1]['y'] = pub_y

pos = improve_graph_pos(TIME_TO_RUN, pos, callback=onupdate, target_refresh_interval=STEP_TIME)

## 3D Graph

In [ ]:
# 3D Graph Display

def pos_to_graphlocs_3d(graph:nx.Graph, pos3d:dict[str,list[int]]):
	nodes = {'x': [], 'y': [], 'z': [], 'l': []}
	for node in graph:
		if node in pos3d:
			x, y, z = pos3d[node]
			nodes['l'].append(node)
			nodes['x'].append(x)
			nodes['y'].append(y)
			nodes['z'].append(z)

	edges = {'x': [], 'y': [], 'z': []}
	for edge in graph.edges():
		if not (edge[0] in pos3d and edge[1] in pos3d): continue
		x0, y0, z0 = pos3d[edge[0]]
		x1, y1, z1 = pos3d[edge[1]]
		
		if len(edges['x']) > 2 and edges['x'][-2] == x0 and edges['y'][-2] == y0 and edges['z'][-2] == z0:
			edges['x'].insert(-1, x1)
			edges['y'].insert(-1, y1)
			edges['z'].insert(-1, z1)
		elif len(edges['x']) > 2 and edges['x'][-2] == x1 and edges['y'][-2] == y1 and edges['z'][-2] == z0:
			edges['x'].insert(-1, x0)
			edges['y'].insert(-1, y0)
			edges['z'].insert(-1, z0)
		else:
			edges['x'].append(x0)
			edges['y'].append(y0)
			edges['z'].append(z0)
			edges['x'].append(x1)
			edges['y'].append(y1)
			edges['z'].append(z1)
			edges['x'].append(None)
			edges['y'].append(None)
			edges['z'].append(None)

	return {
		'nodes': nodes,
		'edges': edges
	}

def init_comparisons_graph(graph:nx.Graph):
	pos3d = nx.spring_layout(MAX_CONNECTED_GRAPH, dim=3, iterations=1)
	loc3d = pos_to_graphlocs_3d(graph, pos3d)

	scatters = []
	# Edges
	scatters.append(go.Scatter3d(
		x=loc3d['edges']['x'], y=loc3d['edges']['y'], z=loc3d['edges']['z'],
		line=dict(
			width=0.3,
			color='#888',
		),
		hoverinfo='none',
		mode='lines',
	))

	# Public Nodes text & colors
	cnct:dict[str,float] = dict()
	mx = graph.number_of_nodes()
	for n1,tgt in nx.all_pairs_shortest_path_length(graph, cutoff=32):
		if not graph.nodes[n1]['public']:
			continue
		ttl = mx
		for n2,ln in tgt.items():
			if n2 == n1: continue
			ttl -= 1/ln
		cnct[n1] = mx/(mx - ttl)

	node_x = []
	node_y = []
	node_z = []
	node_colors = []
	node_text = []
	for i,node in enumerate(loc3d['nodes']['l']):
		if not graph.nodes[node]['public']:
			continue
		deg_pub = graph.degree[node] if node in graph else 0
		deg_prv = graph.degree[node]
		node_x.append(loc3d['nodes']['x'][i])
		node_y.append(loc3d['nodes']['y'][i])
		node_z.append(loc3d['nodes']['z'][i])
		node_text.append(
			f"{node}<br>"
			+ f"{deg_pub} public comparisons<br>"
			+ (f"{deg_prv - deg_pub} private comparisons<br>" if deg_prv > deg_pub else "<br>")
			+ f"Average distance: {cnct[node]:.1f}"
		)
		node_colors.append(cnct[node])

	# Public nodes
	scatters.append(go.Scatter3d(
		x=node_x, y=node_y, z=node_z,
		mode='markers',
		hoverinfo='text',
		marker=dict(
			colorscale='Portland',
			colorbar=dict(
				title='Avg. distance<br>to others',
				tickformat='.1f',
				tickvals=[min(node_colors), *range(int(min(node_colors)+1), int(max(node_colors)+1)), max(node_colors)],
				ticks='outside',
				tickmode='array',
			),
			reversescale=False,
			color=node_colors,
			size=3,
			line=dict(width=0),
		),
		text=node_text,
	))

	fig = go.FigureWidget(data=scatters,
		layout=go.Layout(
			showlegend=False,
			hovermode='closest',
			margin=dict(b=0,l=0,r=0,t=0),
			height=720,
			width=1080,
		),
	)

	fig.update_layout(scene=dict(
		xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
		yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
		zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
	))

	# Fix aspect ratio
	fig.update_yaxes(scaleanchor="x", scaleratio=1)

	return fig,pos3d

fig,pos3d = init_comparisons_graph(private_undgraph)
fig # Display graph (should be the last line of the notebook cell)

In [ ]:
# Untangle 3D Graph above (may run multiple time if needed - Takes time)
TIME_TO_RUN = 300 # Seconds, the longer the prettier
STEP_TIME = 10 # Seconds

def improve_graph_pos3d(time_to_run:int, pos3d:dict[str,list[float]], callback=None, target_refresh_interval:int=None):
	start = time.time()

	target_total_it_count=math.ceil(time_to_run/target_refresh_interval) if target_refresh_interval > 0 else 10
	iterations_count=1
	total_iterations=0
	loops_count = 0
	timer_a = time.time()
	while timer_a - start < time_to_run:
		loops_count += 1
		# Move nodes towards eachother if connected, move them apart from eachother if not connected
		pos3d = nx.spring_layout(MAX_CONNECTED_GRAPH, dim=3, pos=pos3d, iterations=iterations_count, center=[0,0,0], weight=None)
		total_iterations += iterations_count
		timer_b = time.time()
		speed = iterations_count / (timer_b-timer_a)
		expected_remaining_iterations = speed * (time_to_run - timer_b + start)
		if callback:
			callback(pos3d)
		print(f"Iterations: {total_iterations}/{total_iterations + expected_remaining_iterations:.0f} -- Time: {timer_b-start:.1f}/{time_to_run}s -- Speed: {speed:.1f}/s")
		next_iteration_count = int(math.ceil(expected_remaining_iterations / (target_total_it_count - loops_count if loops_count < target_total_it_count else 1)))
		if loops_count > target_total_it_count or next_iteration_count > iterations_count*2 and loops_count > 1:
			# Spring Layout may stop iterating if found an equilibrium. Try to detect this event and stop before max_duration
			break
		# Prepare next iteration
		iterations_count = next_iteration_count
		timer_a = timer_b

	return pos3d

def onupdate3d(pos3d):
	loc = pos_to_graphlocs_3d(private_undgraph, pos3d)
	with fig.batch_update():
		fig.data[0]['x'] = loc['edges']['x']
		fig.data[0]['y'] = loc['edges']['y']
		fig.data[0]['z'] = loc['edges']['z']

		zipped = zip(loc['nodes']['l'], loc['nodes']['x'], loc['nodes']['y'], loc['nodes']['z'])
		pub_l,pub_x,pub_y,pub_z = zip(*filter(lambda z:private_undgraph.nodes[z[0]]['public'], zipped))
		fig.data[1]['x'] = pub_x
		fig.data[1]['y'] = pub_y
		fig.data[1]['z'] = pub_z

pos3d = improve_graph_pos3d(TIME_TO_RUN, pos3d, callback=onupdate3d, target_refresh_interval=STEP_TIME)

## Springs

In [ ]:
# Get all private&public comparisons

cmps:dict[str,dict[str,set[str],set[str],set[str]]] = {}
scores:dict[str,int] = {}
t_scores:dict[str,dict[str,any]] = {} # channels & tags

for c in comparisons:
	cp = [s['score'] for s in c['criteria_scores'] if s['criteria'] == 'largely_recommended'][0]
	cmps.setdefault(c['entity_a'], {'+':set(), '=':set(), '-':set()})['+' if cp > 0 else ('=' if cp == 0 else '-')].add(c['entity_b'])
	cmps.setdefault(c['entity_b'], {'+':set(), '=':set(), '-':set()})['+' if cp < 0 else ('=' if cp == 0 else '-')].add(c['entity_a'])

# Find videos having only positive (ignore =0) recoms => fix them to y=1
# Find videos having only negative (ignore =0) recoms => fix them to y=-1
for vid in list(cmps):
	t_scores.setdefault('channel:'+get(videos[vid], '???', 'entity', 'metadata', 'channel_id'), {'score':0, 'std':0, 'vids':set()})['vids'].add(vid)
	for t in get(videos[vid], [], 'entity', 'metadata', 'tags'):
		t_scores.setdefault('tag:'+t, {'score':0, 'vids':set()})['vids'].add(vid)

	if len(cmps[vid]['-']) + len(cmps[vid]['=']) == 0: # all positive
		scores[vid] = 100.0
	elif len(cmps[vid]['+']) + len(cmps[vid]['=']) == 0: # all negative
		scores[vid] = -100.0
	else: # mixed positives & negatives
		scores[vid] = (get_individual_score(videos[vid]) or 0)

# Remove from t_score when only single element
print('Tags (all):', len(t_scores))
for t in list(t_scores):
	if len(t_scores[t]['vids']) <= 2:
		t_scores.pop(t, None)

# Merge tags having same vids
print('Tags (no singles):', len(t_scores))
stscors = sorted(t_scores)
for i1 in range(len(stscors)-1,0,-1):
	to_rm = False
	for i2 in range(i1):
		if t_scores[stscors[i1]]['vids'] == t_scores[stscors[i2]]['vids']:
			to_rm = True
			break
	if to_rm:
		t_scores.pop(stscors[i1], None)
print('Tags (deduplicated):', len(t_scores))

# Put all others videos to y=0
# Then repeat for some time:
# - For all these other videos, y = average of:
#     - Min score of all videos compared higher
#     - Max score of all videos compared lower
#     - All scores of videos compared same
starttime = time.time()
lastprint = starttime
stoptime = starttime + 60 # seconds
loops = 0
while time.time() < stoptime:
	# Update channel scores
	for tag,cdata in t_scores.items():
		vscores = [scores[v] for v in cdata['vids']]
		cdata['score'] = np.average(vscores)
		cdata['std'] = np.std(vscores)

	newscores: dict[str,int] = {}
	for vid in scores:
		cnt_plus = len(cmps[vid]['+'])
		cnt_mnus = len(cmps[vid]['-'])
		eq = [scores[v2] for v2 in cmps[vid]['=']]

		# Counting tags
		tags = ['channel:'+get(videos[vid], '???', 'entity', 'metadata', 'channel_id'), *('tag:'+t for t in get(videos[vid], [], 'entity', 'metadata', 'tags'))]
		tw_tot = 0
		ts_tot = 0
		tw = [len(t_scores[t]['vids'])/t_scores[t]['std'] for t in tags if t in t_scores and t_scores[t]['std'] > 0]
		tw_sum = sum(tw)
		if tw_sum > 0:
			ts = [t_scores[t]['score'] * len(t_scores[t]['vids'])/t_scores[t]['std'] for t in tags if t in t_scores and t_scores[t]['std'] > 0]
			tw_tot = 1
			ts_tot = tw_tot * sum(ts) / tw_sum

		if cnt_plus > 0 and cnt_mnus > 0: # Mixed '+' & '-' (possibly also '=') comparisons
			min_better = min(scores[v2] for v2 in cmps[vid]['+'])
			max_worst = max(scores[v2] for v2 in cmps[vid]['-'])
			newscores[vid] = (sum(eq) + min_better + max_worst + ts_tot) / (len(eq) + tw_tot + 2)
		elif cnt_plus > 0 and len(eq) == 0: # only '+' (no '=')
			min_better = min(scores[v2] for v2 in cmps[vid]['+'])
			newscores[vid] = (min_better + ts_tot - 99) / (tw_tot + 2)
		elif cnt_mnus > 0 and len(eq) == 0: # only '-' (no '=')
			max_worst = max(scores[v2] for v2 in cmps[vid]['-'])
			newscores[vid] = (max_worst + ts_tot + 101) / (tw_tot + 2)
		else: # only '+' or only '-', and also '=' comparisons
			newscores[vid] = (sum(eq) - cnt_plus + cnt_mnus + ts_tot) / (len(eq) + cnt_plus + cnt_mnus + tw_tot)

	scores = newscores
	loops += 1
	now = time.time()
	if lastprint <= now - 2.5:
		print(f"{loops:4d} loops [{min(scores.values()):+.6f}, {sum(scores.values()):+.6f}, {max(scores.values()):+.6f}]")
		lastprint = now
print(f"{loops:4d} loops [{min(scores.values()):+.6f}, {sum(scores.values()):+.6f}, {max(scores.values()):+.6f}]")

mmin = min(scores.values())
mmax = max(scores.values())
def nrm(value):
	return (value-mmin)/(mmax-mmin)*200-100

print()
def ptxt(cmps):
	str = ' '.join(f"{nrm(scores[c]):+.0f}" for c in sorted(cmps, key=scores.get))
	return '(' + str + ')'
for vid in sorted(scores, key=scores.get, reverse=True):
	# Check if public
	if private_undgraph.has_node(vid):
		print(vid, f"{nrm(scores[vid]):+6.2f} : -{ptxt(cmps[vid]['-'])} ={ptxt(cmps[vid]['='])} +{ptxt(cmps[vid]['+'])}")

In [ ]:
labels = []
cc = []
xx = []
yy = []
for s in sorted(scores, key=lambda x: videos[x]['individual_rating']['n_comparisons']):
	labels.append(TOURNESOL_API.prettyPrintVData(videos[s]))
	#cc.append(videos[s]['individual_rating']['n_comparisons'])
	cc.append(videos[s]['individual_rating']['is_public'])
	xx.append(get_individual_score(videos[s]))
	yy.append(nrm(scores[s]))

fig = px.scatter(x=xx, y=yy, color=cc, hover_name=labels, labels={'x': 'Tournesol indiv score', 'y': 'Estimated value', 'color': 'Public'}, marginal_x="histogram", marginal_y="histogram")
fig.update_layout(showlegend=False, width=900, height=900)
fig.layout.xaxis.domain = (fig.layout.xaxis.domain[0], 0.9)
fig.layout.yaxis.domain = (fig.layout.yaxis.domain[0], 0.9)
fig.layout.xaxis2.domain = (fig.layout.xaxis.domain[1], fig.layout.xaxis2.domain[1])
fig.layout.yaxis2.domain = (fig.layout.yaxis2.domain[0], fig.layout.yaxis.domain[1])
fig.layout.xaxis3.domain = (fig.layout.xaxis3.domain[0], fig.layout.xaxis.domain[1])
fig.layout.yaxis3.domain = (fig.layout.yaxis.domain[1], fig.layout.yaxis3.domain[1])
fig.layout.xaxis4.domain = (fig.layout.xaxis.domain[1], fig.layout.xaxis4.domain[1])
fig.layout.yaxis4.domain = (fig.layout.yaxis.domain[1], fig.layout.yaxis4.domain[1])
fig.show()

In [ ]:
for tag in sorted(t_scores, key=lambda c: t_scores[c]['score']*len(t_scores[c]['vids'])/t_scores[c]['std'], reverse=True):
	cs = t_scores[tag]
	if len(cs['vids']) < 2:
		continue
	pretty_tag = tag if tag[:4] == 'tag: ' else 'Channel: ' + videos[list(cs['vids'])[0]]['entity']['metadata']['uploader']
	print(f"avg:{nrm(cs['score']):+4.0f} - vids:{len(cs['vids']):4d} - w:{len(cs['vids'])/cs['std']:5.0f} - {pretty_tag}")